In [1]:
import sqlite3
import pandas as pd


In [2]:
conn = sqlite3.connect('MusicPlayer.db') 

In [3]:
sql_query = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Interactions
                               ''', conn)

In [4]:
df = pd.DataFrame(sql_query, columns = ['id', 'user_id', 'song_id' , 'like' , 'listen_count'])

In [5]:
print (df)


        id  user_id  song_id  like  listen_count
0        1        1       63     1           933
1        2        1       70     1           830
2        3        1       18     1           241
3        4        1       37     1           803
4        5        1       56     1           778
...    ...      ...      ...   ...           ...
3115  3116      156      378     1            38
3116  3117      156      516     1           438
3117  3118      156      403     1           817
3118  3119      156      539     1           778
3119  3120      156      435     1           989

[3120 rows x 5 columns]


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import time 
import joblib
import app
from app import Interactions, db
from sqlalchemy import create_engine
import model.Recommenders as Recommenders
import model.Evaluation as Evaluation


pygame 2.0.2 (SDL 2.0.16, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
sql_query = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Songs
                               ''', conn)

In [8]:
song_df = pd.DataFrame(sql_query, columns = ['id', 'name', 'path' , 'artist' , 'genre' , 'cover_photo' , 'duration' , 'total_likes' , 'total_listen_count'])

In [9]:
song_df.head()

,id,name,path,artist,genre,cover_photo,duration,total_likes,total_listen_count
0,1,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,4,2339
1,2,Chakdait Chhora.mp3,Dataset\garhwali\Anil - Chakdait Chhora.mp3,Anil,garhwali,/static/images/garhwali.jpeg,5:39,14,7006
2,3,Jawa Pardesh.mp3,Dataset\garhwali\Anil - Jawa Pardesh.mp3,Anil,garhwali,/static/images/garhwali.jpeg,6:19,9,3677
3,4,Kan Full Khilana.mp3,Dataset\garhwali\Anil - Kan Full Khilana.mp3,Anil,garhwali,/static/images/garhwali.jpeg,9:44,5,2425
4,5,Latuli Teri Mukhdi.mp3,Dataset\garhwali\Anil - Latuli Teri Mukhdi.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:56,4,2464


In [10]:
song_df.head()

,id,name,path,artist,genre,cover_photo,duration,total_likes,total_listen_count
0,1,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,4,2339
1,2,Chakdait Chhora.mp3,Dataset\garhwali\Anil - Chakdait Chhora.mp3,Anil,garhwali,/static/images/garhwali.jpeg,5:39,14,7006
2,3,Jawa Pardesh.mp3,Dataset\garhwali\Anil - Jawa Pardesh.mp3,Anil,garhwali,/static/images/garhwali.jpeg,6:19,9,3677
3,4,Kan Full Khilana.mp3,Dataset\garhwali\Anil - Kan Full Khilana.mp3,Anil,garhwali,/static/images/garhwali.jpeg,9:44,5,2425
4,5,Latuli Teri Mukhdi.mp3,Dataset\garhwali\Anil - Latuli Teri Mukhdi.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:56,4,2464


In [11]:
song_df.rename(columns = {'id':'song_id'}, inplace = True)
song_df.head()

,song_id,name,path,artist,genre,cover_photo,duration,total_likes,total_listen_count
0,1,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,4,2339
1,2,Chakdait Chhora.mp3,Dataset\garhwali\Anil - Chakdait Chhora.mp3,Anil,garhwali,/static/images/garhwali.jpeg,5:39,14,7006
2,3,Jawa Pardesh.mp3,Dataset\garhwali\Anil - Jawa Pardesh.mp3,Anil,garhwali,/static/images/garhwali.jpeg,6:19,9,3677
3,4,Kan Full Khilana.mp3,Dataset\garhwali\Anil - Kan Full Khilana.mp3,Anil,garhwali,/static/images/garhwali.jpeg,9:44,5,2425
4,5,Latuli Teri Mukhdi.mp3,Dataset\garhwali\Anil - Latuli Teri Mukhdi.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:56,4,2464


In [12]:
merged_song_df = pd.merge(df, song_df.drop_duplicates(['song_id']), on="song_id", how="left")

In [13]:
merged_song_df.head()

,id,user_id,song_id,like,listen_count,name,path,artist,genre,cover_photo,duration,total_likes,total_listen_count
0,1,1,63,1,933,Mor Moriki Maand Pelya.mp3,Dataset\garhwali\Narendra Singh Negi - Mor Mor...,Narendra Singh Negi,garhwali,/static/images/garhwali.jpeg,7:25,4,2035
1,2,1,70,1,830,Rai Maasi Ko Phool.mp3,Dataset\garhwali\Narendra Singh Negi - Rai Maa...,Narendra Singh Negi,garhwali,/static/images/garhwali.jpeg,6:32,5,2960
2,3,1,18,1,241,Kya Dan Lagdi Tu.mp3,Dataset\garhwali\Gajendra Rana - Kya Dan Lagdi...,Gajendra Rana,garhwali,/static/images/garhwali.jpeg,4:55,5,1764
3,4,1,37,1,803,Bavada Re Bavada Re.mp3,Dataset\garhwali\Narender Singh Negi - Bavada ...,Narender Singh Negi,garhwali,/static/images/garhwali.jpeg,7:28,5,3494
4,5,1,56,1,778,Ghar Bati Chitthi Aegyayee.mp3,Dataset\garhwali\Narendra Singh Negi - Ghar Ba...,Narendra Singh Negi,garhwali,/static/images/garhwali.jpeg,5:56,8,4744


In [14]:
merged_song_df.sort_values(by='song_id')

,id,user_id,song_id,like,listen_count,name,path,artist,genre,cover_photo,duration,total_likes,total_listen_count
1520,1521,77,1,1,625,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,4,2339
62,63,4,1,1,790,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,4,2339
61,62,4,1,1,154,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,4,2339
2256,2257,113,1,1,770,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,4,2339
1541,1542,78,2,1,557,Chakdait Chhora.mp3,Dataset\garhwali\Anil - Chakdait Chhora.mp3,Anil,garhwali,/static/images/garhwali.jpeg,5:39,14,7006
...,...,...,...,...,...,...,...,...,...,...,...,...,...
768,769,39,562,1,729,Jai Shiv Shankar Jai Mahadeva.mp3,Dataset\bhajan\Ravindra Jain - Jai Shiv Shanka...,Ravindra Jain,bhajan,/static/images/bhajan.jpeg,18:24,13,7082
400,401,21,562,1,853,Jai Shiv Shankar Jai Mahadeva.mp3,Dataset\bhajan\Ravindra Jain - Jai Shiv Shanka...,Ravindra Jain,bhajan,/static/images/bhajan.jpeg,18:24,13,7082
3084,3085,155,562,1,300,Jai Shiv Shankar Jai Mahadeva.mp3,Dataset\bhajan\Ravindra Jain - Jai Shiv Shanka...,Ravindra Jain,bhajan,/static/images/bhajan.jpeg,18:24,13,7082
401,402,21,562,1,481,Jai Shiv Shankar Jai Mahadeva.mp3,Dataset\bhajan\Ravindra Jain - Jai Shiv Shanka...,Ravindra Jain,bhajan,/static/images/bhajan.jpeg,18:24,13,7082


In [15]:
len(merged_song_df)

3120

In [16]:
#Creating a subset of our dataset

In [17]:
ong_df =ong_df = merged_song_df.head(100)

In [18]:
print(ong_df)

     id  user_id  song_id  like  listen_count                            name  \
0     1        1       63     1           933      Mor Moriki Maand Pelya.mp3   
1     2        1       70     1           830          Rai Maasi Ko Phool.mp3   
2     3        1       18     1           241            Kya Dan Lagdi Tu.mp3   
3     4        1       37     1           803         Bavada Re Bavada Re.mp3   
4     5        1       56     1           778  Ghar Bati Chitthi Aegyayee.mp3   
..  ...      ...      ...   ...           ...                             ...   
95   96        5      334     1           803              Swag Mera Desi.mp3   
96   97        5      444     1            21                  Joban Duno.mp3   
97   98        5       55     1           336        Bhandi Dinu Ma Dikhe.mp3   
98   99        5      206     1           433          Latthay Di Chaadar.mp3   
99  100        5       21     1           606                Pushpa Chori.mp3   

                           

In [22]:
# song_grouped = merged_song_df.groupby([‘song’]).agg({‘listen_count’: ‘count’}).reset_index()

In [23]:
train_data, test_data = train_test_split(merged_song_df, test_size = 0.20, random_state=0)

In [24]:
print(train_data.head(5))

        id  user_id  song_id  like  listen_count  \
867    868       44      170     1           429   
2838  2839      142      478     1           632   
1756  1757       88      218     1           579   
2842  2843      143      261     1           661   
682    683       35       59     1           392   

                                    name  \
867                 Kabhi Yun Bhi Aa.mp3   
2838                 Kamkarat Naikhe.mp3   
1756  Uddi Ja (Coke Studio Season 9).mp3   
2842               Pori Pathi Vizhum.mp3   
682                Hari Tori Kakhari.mp3   

                                                   path               artist  \
867    Dataset\ghazal\Talat Aziz - Kabhi Yun Bhi Aa.mp3           Talat Aziz   
2838  Dataset\bhojpuri\Pradum Lal - Kamkarat Naikhe.mp3           Pradum Lal   
1756  Dataset\sufi\Mohsin Abbas Haider - Uddi Ja (Co...  Mohsin Abbas Haider   
2842  Dataset\bollywood_rap\Anirudh Ravichander - Po...  Anirudh Ravichander   
682   Dataset\garhwali

In [25]:
pm = Recommenders.popularity_recommender_py()

In [26]:
pm.create(train_data, 'user_id', 'name')

In [27]:
user_id = 1

In [28]:
pm.recommend(user_id)

,user_id,name,score,Rank
174,1,Hara Hu Sath Nibhao Na Baba Haare Haare 3.mp3,14,1.0
348,1,O Paalanhaare.mp3,14,2.0
213,1,Jai Shiv Omkara.mp3,13,3.0
244,1,Kanha Teri Murli Ki.mp3,13,4.0
16,1,Ae Malik Tere Bande Hum.mp3,12,5.0
97,1,Dam Mast Qalandar.mp3,12,6.0
122,1,Dost.mp3,12,7.0
319,1,Meri Maiya Ne.mp3,12,8.0
429,1,Sancha Naam Tera,12,9.0
78,1,Chakdait Chhora.mp3,11,10.0


In [29]:
pm.recommend(9)

,user_id,name,score,Rank
174,9,Hara Hu Sath Nibhao Na Baba Haare Haare 3.mp3,14,1.0
348,9,O Paalanhaare.mp3,14,2.0
213,9,Jai Shiv Omkara.mp3,13,3.0
244,9,Kanha Teri Murli Ki.mp3,13,4.0
16,9,Ae Malik Tere Bande Hum.mp3,12,5.0
97,9,Dam Mast Qalandar.mp3,12,6.0
122,9,Dost.mp3,12,7.0
319,9,Meri Maiya Ne.mp3,12,8.0
429,9,Sancha Naam Tera,12,9.0
78,9,Chakdait Chhora.mp3,11,10.0


In [30]:
is_model = Recommenders.item_similarity_recommender_py()

In [31]:
is_model.create(train_data, 'user_id', 'name')

In [32]:
user_id=7

In [33]:
user_items = is_model.get_user_items(user_id)

In [34]:
for user_item in user_items:
    print(user_item)

Gaon Ku Baatu.mp3
Hai Teri Rumaala.mp3
Babu Fasali.mp3
Jab Jab Bhatar Kare.mp3
Meri Maiya Ne.mp3
Akhilesh Bhaiya.mp3
Ae Malik Tere Bande Hum.mp3
Death Route.mp3
Mela Ki Chori.mp3
Ramji Ke Naam Ne To Pathar Bhi Tare.mp3
Milal Bate Mauga Bhatar.mp3
Pushpa Chori.mp3
Sukh Ke Sab Sathi.mp3
Kajrani Aakhi Teri.mp3
Khatu Ji Jaane Ko Jee Lalchata Hai.mp3
Mor Moriki Maand Pelya.mp3
Chumma Chapkauwa.mp3


In [35]:
is_model.recommend(user_id)

No. of unique songs for the user: 17
no. of unique songs in the training set: 545
Non zero values in cooccurence_matrix :1522


,user_id,song,score,rank
0,7,O Paalanhaare.mp3,0.044899,1
1,7,Shaam Se Aankh Mein Nami Si Hai.mp3,0.043665,2
2,7,Kanha Teri Murli Ki (1).mp3,0.042931,3
3,7,Rangli Rangli Chunariya Tere Naam Balaji.mp3,0.040804,4
4,7,Jai Shiv Omkara.mp3,0.040637,5
5,7,Khubsurat Hai Aankhen.mp3,0.040107,6
6,7,Mila Mujhe Teri Kripa Se (1).mp3,0.039816,7
7,7,Shayad (From Love Aaj Kal).mp3,0.038971,8
8,7,Parsi Dekhi Chhajjam Chhai.mp3,0.038348,9
9,7,Jag Soona Soona Lage.mp3,0.037656,10


In [36]:
is_model.get_similar_items(['Saiyaan'])

no. of unique songs in the training set: 545
Non zero values in cooccurence_matrix :0


,user_id,song,score,rank
0,,Pana Piremi.mp3,0.0,1
1,,Photo.mp3,0.0,2
2,,She Move It Like.mp3,0.0,3
3,,Aap Agar In Dinon Yahan Hote.mp3,0.0,4
4,,Kakh Harchi Hey Deedo.mp3,0.0,5
5,,Aao Milo Chalo.mp3,0.0,6
6,,Tu Ambar Ki Aankh Ka Tara.mp3,0.0,7
7,,Rai Maasi Ko Phool.mp3,0.0,8
8,,Mere Liye Tum Kaafi Ho (From Shubh Mangal Zyad...,0.0,9
9,,Tujhe Kitna Chahne Lage (From Kabir Singh) (1)...,0.0,10


In [37]:
is_model.get_similar_items(['Kun Faya Kun'])

no. of unique songs in the training set: 545
Non zero values in cooccurence_matrix :0


,user_id,song,score,rank
0,,Pana Piremi.mp3,0.0,1
1,,Photo.mp3,0.0,2
2,,She Move It Like.mp3,0.0,3
3,,Aap Agar In Dinon Yahan Hote.mp3,0.0,4
4,,Kakh Harchi Hey Deedo.mp3,0.0,5
5,,Aao Milo Chalo.mp3,0.0,6
6,,Tu Ambar Ki Aankh Ka Tara.mp3,0.0,7
7,,Rai Maasi Ko Phool.mp3,0.0,8
8,,Mere Liye Tum Kaafi Ho (From Shubh Mangal Zyad...,0.0,9
9,,Tujhe Kitna Chahne Lage (From Kabir Singh) (1)...,0.0,10


In [38]:
is_model.get_similar_items(['Mere Gully Mein (feat Naezy).mp3'])

no. of unique songs in the training set: 545
Non zero values in cooccurence_matrix :30


,user_id,song,score,rank
0,,Jungli Sher.mp3,0.333333,1
1,,Nazere Mili.mp3,0.250000,2
2,,Ghungroo (From War).mp3,0.250000,3
3,,Ve Maahi.mp3,0.250000,4
4,,Proper Patola.mp3,0.250000,5
5,,Dawood.mp3,0.200000,6
6,,Lad Gail Najariya.mp3,0.200000,7
7,,Surili Akhiyon Wale (From Veer).mp3,0.200000,8
8,,Bijlee.mp3,0.200000,9
9,,Ail Bade Saiya.mp3,0.200000,10


## Quantitative comparison between the models¶
We now formally compare the popularity and the personalized models using precision-recall curves.

Class to calculate precision and recall (This can be used as a black box)

In [39]:
#Evaluation.precision_recall_calculator

# Matrix Factorization based Recommender System

Using SVD matrix factorization based collaborative filtering recommender system -------------------------------------------------------------------------------- The following code implements a Singular Value Decomposition (SVD) based matrix factorization collaborative filtering recommender system. The user ratings matrix used is a small matrix as follows: Item0 Item1 Item2 Item3 User0 3 1 2 3 User1 4 3 4 3 User2 3 2 1 5 User3 1 6 5 2 User4 0 0 5 0 As we can see in the above matrix, all users except user 4 rate all items. The code calculates predicted recommendations for user 4

In [40]:

#Code source written with help from: 
#http://antoinevastel.github.io/machine%20learning/python/2016/02/14/svd-recommender-system.html

import math as mt
import csv
from sparsesvd import sparsesvd #used for matrix factorization
import numpy as np
from scipy.sparse import csc_matrix #used for sparse matrix
from scipy.sparse.linalg import * #used for matrix multiplication

#Note: You may need to install the library sparsesvd. Documentation for 
#sparsesvd method can be found here:
#https://pypi.python.org/pypi/sparsesvd/

In [41]:
#constants defining the dimensions of our User Rating Matrix (URM)
MAX_PID = 4
MAX_UID = 5

#Compute SVD of the user ratings matrix
def computeSVD(urm, K):
    U, s, Vt = sparsesvd(urm, K)

    dim = (len(s), len(s))
    S = np.zeros(dim, dtype=np.float32)
    for i in range(0, len(s)):
        S[i,i] = mt.sqrt(s[i])

    U = csc_matrix(np.transpose(U), dtype=np.float32)
    S = csc_matrix(S, dtype=np.float32)
    Vt = csc_matrix(Vt, dtype=np.float32)
    
    return U, S, Vt

#Compute estimated rating for the test user
def computeEstimatedRatings(urm, U, S, Vt, uTest, K, test):
    rightTerm = S*Vt 

    estimatedRatings = np.zeros(shape=(MAX_UID, MAX_PID), dtype=np.float16)
    for userTest in uTest:
        prod = U[userTest, :]*rightTerm
        #we convert the vector to dense format in order to get the indices 
        #of the movies with the best estimated ratings 
        estimatedRatings[userTest, :] = prod.todense()
        recom = (-estimatedRatings[userTest, :]).argsort()[:250]
    return recom


In [42]:
#Used in SVD calculation (number of latent factors)
K=2

#Initialize a sample user rating matrix
urm = np.array([[3, 1, 2, 3],[4, 3, 4, 3],[3, 2, 1, 5], [1, 6, 5, 2], [5, 0,0 , 0]])
urm = csc_matrix(urm, dtype=np.float32)

#Compute SVD of the input user ratings matrix
U, S, Vt = computeSVD(urm, K)

#Test user set as user_id 4 with ratings [0, 0, 5, 0]
uTest = [4]
print("User id for whom recommendations are needed: %d" % uTest[0])

#Get estimated rating for test user
print("Predictied ratings:")
uTest_recommended_items = computeEstimatedRatings(urm, U, S, Vt, uTest, K, True)
print(uTest_recommended_items)

User id for whom recommendations are needed: 4
Predictied ratings:
[0 3 2 1]
